<a href="https://colab.research.google.com/github/jintubhuyan-2000/ML-XAI_ForestFire/blob/main/Fire_Risk_RF_Model_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown --quiet

# Use gdown to download the folder
!gdown "https://drive.google.com/drive/u/0/folders/1p-hZYcWiCIoc8Y0LRUPtqJJGe9_PjlFN" --folder


Retrieving folder contents
Processing file 1JLTkxujzAOyY8SPMLsua_QAED6_0f3yl FireRisk_Classes_RF__forest.csv
Processing file 160nsutuW5crSeTbEhh9oAoA0HJa8n_cp FireRisk_Classes_RF__forest.png
Processing file 10LD44_avGgopWeHNYxF2km35VpAjtDMS FireRisk_Classes_RF__grassland.csv
Processing file 12_ewAIDHNIHp8xbSXDPGLJSPlAXialbV FireRisk_Classes_RF__grassland.png
Processing file 1fwjdaRxAd07vDJ0Y7yaHPDbG-Exu2v6Q FireRisk_Classes_RF__grassland.tif
Processing file 1acArgVbiC6laYQAJ-rGs6sCuXJ1PRZQ5 FireRisk_Classes_RF__grassland.tif.ovr
Processing file 1kGbfN4DNVY5Skn_LAKFrzB9_hjEBTlkl FireRisk_Classes_RF__grassland.tif.vat.cpg
Processing file 1HDP-h-AQVTgsXWv2vwR6qaBNEjeohFjN FireRisk_Classes_RF__grassland.tif.vat.dbf
Processing file 1ahvmIzLK2JHcJhULl0z1c2RPP5ZqpGSl FireRisk_Classes_RF__grassland.tif.vat.dbf.JINTUMONIBHUYAN.25792.17896.sr.lock
Processing file 1FXqqZBfK6idJjAiFFYlwGKVQVuEo11jy FireRisk_Classes_RF_Forest.tif
Processing file 18jJuSulidTtDhiaVveC9DutEoj09DXI8 FireRisk_Classes_RF

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
import os

# ================================
# 📁 Define paths
# ================================
base_path = "/content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Performance Data"
save_dir = "/content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Results"
os.makedirs(save_dir, exist_ok=True)

# 📄 Load ONLY RF CSV
df_rf = pd.read_csv(os.path.join(base_path, "FireRisk_Validation_RF_AUC_forest.csv"))

# 🔧 Classification threshold
threshold = 0.5

# ================================
# 📊 Evaluate Random Forest
# ================================
def evaluate_rf(df, model_name="Random Forest", threshold=0.5, save_dir="."):
    y_true = df['class']
    y_prob = df['probability']
    y_pred = (y_prob >= threshold).astype(int)

    # 🔹 Metrics
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\n📊 {model_name} Performance:")
    print(f"AUC: {auc:.3f}, Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
    print(classification_report(y_true, y_pred, target_names=["No Fire", "Fire"]))

    # 🔹 Confusion Matrix
    plt.figure(figsize=(6,5))
    sns.heatmap(confusion_matrix(y_true, y_pred),
                annot=True, fmt='d', cmap='Blues',
                xticklabels=["No Fire", "Fire"],
                yticklabels=["No Fire", "Fire"])
    plt.title(f"{model_name} - Confusion Matrix", fontsize=16, fontweight='bold')
    plt.xlabel("Predicted", fontsize=16)
    plt.ylabel("Actual", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    cm_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_Confusion_Matrix.png")
    plt.savefig(cm_path, dpi=300)
    plt.close()
    print(f"✅ Saved confusion matrix to: {cm_path}")

    return y_true.values, y_prob.values, auc

# ================================
# 🎨 ROC with Bootstrapped Ribbon
# ================================
def plot_roc_with_ci(y_true, y_prob, model_name="Random Forest", save_dir=".", n_bootstraps=1000):
    rng = np.random.RandomState(42)
    fpr_grid = np.linspace(0, 1, 100)

    tprs = []
    aucs = []

    # Bootstrapping
    for i in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            continue
        y_true_boot = y_true[indices]
        y_prob_boot = y_prob[indices]

        fpr, tpr, _ = roc_curve(y_true_boot, y_prob_boot)
        tpr_interp = np.interp(fpr_grid, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc_score(y_true_boot, y_prob_boot))

    tprs = np.array(tprs)
    mean_tpr = tprs.mean(axis=0)
    std_tpr = tprs.std(axis=0)

    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)

    # Upper and lower bound
    tpr_upper = np.minimum(mean_tpr + std_tpr, 1)
    tpr_lower = np.maximum(mean_tpr - std_tpr, 0)

    # Plot
    plt.figure(figsize=(7,6))
    plt.plot(fpr_grid, mean_tpr, color='darkgreen', lw=2,
             label=f"Mean ROC (AUC = {mean_auc:.3f} ± {std_auc:.3f})")
    plt.fill_between(fpr_grid, tpr_lower, tpr_upper, color='green', alpha=0.2, label='±1 SD')
    plt.plot([0,1],[0,1], 'k--', lw=1, label='No Skill')

    # Styling
    plt.xlabel("False Positive Rate", fontsize=18)
    plt.ylabel("True Positive Rate", fontsize=16)
    plt.title(f"ROC Curve with Confidence Interval - {model_name}", fontsize=18, fontweight='bold')
    plt.legend(loc="lower right", fontsize=18)
    plt.grid(alpha=0.3)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()

    # Save
    roc_ci_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_ROC_with_CI.png")
    plt.savefig(roc_ci_path, dpi=300)
    plt.close()
    print(f"✅ Saved ROC curve with ribbon to: {roc_ci_path}")


# ================================
# 🚀 Run
# ================================
y_true_rf, y_prob_rf, auc_rf = evaluate_rf(df_rf, save_dir=save_dir)
plot_roc_with_ci(y_true_rf, y_prob_rf, save_dir=save_dir)
# ================================
# 📊 Evaluate Random Forest
# ================================
def evaluate_rf(df, model_name="Random Forest", threshold=0.5, save_dir="."):
    y_true = df['class']
    y_prob = df['probability']
    y_pred = (y_prob >= threshold).astype(int)

    # 🔹 Metrics
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\n📊 {model_name} Performance:")
    print(f"AUC: {auc:.3f}, Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
    print(classification_report(y_true, y_pred, target_names=["No Fire", "Fire"]))

    # 🔹 Confusion Matrix
    plt.figure(figsize=(6,5))
    sns.heatmap(confusion_matrix(y_true, y_pred),
                annot=True, fmt='d', cmap='Blues',
                xticklabels=["No Fire", "Fire"],
                yticklabels=["No Fire", "Fire"])
    plt.title(f"{model_name} - Confusion Matrix", fontsize=16, fontweight='bold')
    plt.xlabel("Predicted", fontsize=18)
    plt.ylabel("Actual", fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    cm_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_Confusion_Matrix.png")
    plt.savefig(cm_path, dpi=300)
    plt.close()
    print(f"✅ Saved confusion matrix to: {cm_path}")

    return y_true.values, y_prob.values, auc

# ================================
# 🎨 ROC with Bootstrapped Ribbon
# ================================
def plot_roc_with_ci(y_true, y_prob, model_name="Random Forest", save_dir=".", n_bootstraps=1000):
    rng = np.random.RandomState(42)
    fpr_grid = np.linspace(0, 1, 100)

    tprs = []
    aucs = []

    # Bootstrapping
    for i in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            continue
        y_true_boot = y_true[indices]
        y_prob_boot = y_prob[indices]

        fpr, tpr, _ = roc_curve(y_true_boot, y_prob_boot)
        tpr_interp = np.interp(fpr_grid, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc_score(y_true_boot, y_prob_boot))

    tprs = np.array(tprs)
    mean_tpr = tprs.mean(axis=0)
    std_tpr = tprs.std(axis=0)

    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)

    # Upper and lower bound
    tpr_upper = np.minimum(mean_tpr + std_tpr, 1)
    tpr_lower = np.maximum(mean_tpr - std_tpr, 0)

    # Plot
    plt.figure(figsize=(7,6))
    plt.plot(fpr_grid, mean_tpr, color='darkgreen', lw=2,
             label=f"Mean ROC (AUC = {mean_auc:.3f} ± {std_auc:.3f})")
    plt.fill_between(fpr_grid, tpr_lower, tpr_upper, color='green', alpha=0.2, label='±1 SD')
    plt.plot([0,1],[0,1], 'k--', lw=1, label='No Skill')

    # Styling
    plt.xlabel("False Positive Rate", fontsize=18)
    plt.ylabel("True Positive Rate", fontsize=18)
    plt.title(f"ROC Curve with Confidence Interval - {model_name}", fontsize=18, fontweight='bold')
    plt.legend(loc="lower right", fontsize=1)
    plt.grid(alpha=0.3)
    plt.xticks(fontsize=1)
    plt.yticks(fontsize=18)
    plt.tight_layout()

    # Save
    roc_ci_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_ROC_with_CI.png")
    plt.savefig(roc_ci_path, dpi=300)
    plt.close()
    print(f"✅ Saved ROC curve with ribbon to: {roc_ci_path}")


# ================================
# 🚀 Run
# ================================
y_true_rf, y_prob_rf, auc_rf = evaluate_rf(df_rf, save_dir=save_dir)
plot_roc_with_ci(y_true_rf, y_prob_rf, save_dir=save_dir)



📊 Random Forest Performance:
AUC: 0.997, Accuracy: 0.949, Precision: 0.930, Recall: 0.997, F1: 0.963
              precision    recall  f1-score   support

     No Fire       0.99      0.86      0.92       976
        Fire       0.93      1.00      0.96      1879

    accuracy                           0.95      2855
   macro avg       0.96      0.93      0.94      2855
weighted avg       0.95      0.95      0.95      2855

✅ Saved confusion matrix to: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Random_Forest_Confusion_Matrix.png
✅ Saved ROC curve with ribbon to: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Random_Forest_ROC_with_CI.png

📊 Random Forest Performance:
AUC: 0.997, Accuracy: 0.949, Precision: 0.930, Recall: 0.997, F1: 0.963
              precision    recall  f1-score   support

     No Fire       0.99      0.86      0.92       976
        Fire       0.93      1.00      0.96      1879

    accuracy                   

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
import os

# ================================
# 📁 Define paths
# ================================
base_path = "/content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Performance Data"
save_dir = "/content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Results"
os.makedirs(save_dir, exist_ok=True)

# 📄 Load ONLY RF CSV
df_rf = pd.read_csv(os.path.join(base_path, "FireRisk_Validation_RF_AUC_forest.csv"))

# 🔧 Classification threshold
threshold = 0.5

# ================================
# 📊 Evaluate Random Forest
# ================================
def evaluate_rf(df, model_name="Random Forest", threshold=0.5, save_dir="."):
    y_true = df['class']
    y_prob = df['probability']
    y_pred = (y_prob >= threshold).astype(int)

    # 🔹 Metrics
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\n📊 {model_name} Performance:")
    print(f"AUC: {auc:.3f}, Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}")
    print(classification_report(y_true, y_pred, target_names=["No Fire", "Fire"]))

    # 🔹 Confusion Matrix
    plt.figure(figsize=(6,5))
    sns.heatmap(confusion_matrix(y_true, y_pred),
                annot=True, fmt='d', cmap='Blues',
                xticklabels=["No Fire", "Fire"],
                yticklabels=["No Fire", "Fire"])
    plt.title(f"{model_name} - Confusion Matrix", fontsize=18, fontweight='bold')
    plt.xlabel("Predicted", fontsize=18)
    plt.ylabel("Actual", fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    cm_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_Confusion_Matrix.png")
    plt.savefig(cm_path, dpi=300)
    plt.close()
    print(f"✅ Saved confusion matrix to: {cm_path}")

    return y_true.values, y_prob.values, auc

# ================================
# 🎨 ROC with Bootstrapped Ribbon
# ================================
def plot_roc_with_ci(y_true, y_prob, model_name="Random Forest", save_dir=".", n_bootstraps=1000):
    rng = np.random.RandomState(42)
    fpr_grid = np.linspace(0, 1, 100)
    tprs, aucs = [], []

    for _ in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:  # skip if only one class
            continue
        y_true_boot = y_true[indices]
        y_prob_boot = y_prob[indices]

        fpr, tpr, _ = roc_curve(y_true_boot, y_prob_boot)
        tpr_interp = np.interp(fpr_grid, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc_score(y_true_boot, y_prob_boot))

    tprs = np.array(tprs)
    mean_tpr = tprs.mean(axis=0)
    std_tpr = tprs.std(axis=0)
    tpr_upper = np.minimum(mean_tpr + std_tpr, 1)
    tpr_lower = np.maximum(mean_tpr - std_tpr, 0)

    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)

    # Plot
    plt.figure(figsize=(7,6))
    plt.plot(fpr_grid, mean_tpr, color='darkgreen', lw=2,
             label=f"Mean ROC (AUC = {mean_auc:.3f} ± {std_auc:.3f})")
    plt.fill_between(fpr_grid, tpr_lower, tpr_upper, color='green', alpha=0.2, label='±1 SD')
    plt.plot([0,1],[0,1], 'k--', lw=1, label='No Skill')

    plt.xlabel("False Positive Rate", fontsize=18)
    plt.ylabel("True Positive Rate", fontsize=18)
    plt.title(f"ROC Curve with Confidence Interval ", fontsize=18, fontweight='bold')
    plt.legend(loc="lower right", fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    roc_ci_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_ROC_with_CI.png")
    plt.savefig(roc_ci_path, dpi=300)
    plt.close()
    print(f"✅ Saved ROC curve with ribbon to: {roc_ci_path}")

# ================================
# 🎨 Precision-Recall Curve with Bootstrapped Ribbon
# ================================
def plot_pr_with_ci(y_true, y_prob, model_name="Random Forest", save_dir=".", n_bootstraps=1000):
    rng = np.random.RandomState(42)
    recall_grid = np.linspace(0, 1, 100)
    precisions = []

    for _ in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            continue
        y_true_boot = y_true[indices]
        y_prob_boot = y_prob[indices]

        precision, recall, _ = precision_recall_curve(y_true_boot, y_prob_boot)
        precision_interp = np.interp(recall_grid, recall[::-1], precision[::-1])
        precisions.append(precision_interp)

    precisions = np.array(precisions)
    mean_precision = precisions.mean(axis=0)
    std_precision = precisions.std(axis=0)
    precision_upper = np.minimum(mean_precision + std_precision, 1)
    precision_lower = np.maximum(mean_precision - std_precision, 0)

    baseline = np.mean(y_true)

    # Plot
    plt.figure(figsize=(7,6))
    plt.plot(recall_grid, mean_precision, color='darkblue', lw=2, label="Mean PR Curve")
    plt.fill_between(recall_grid, precision_lower, precision_upper, color='blue', alpha=0.2, label='±1 SD')
    plt.hlines(baseline, 0, 1, color='gray', linestyle='--', label='No Skill')

    plt.xlabel("Recall", fontsize=18)
    plt.ylabel("Precision", fontsize=18)
    plt.title(f"Precision-Recall Curve with Confidence Interval", fontsize=18, fontweight='bold')
    plt.legend(loc="lower left", fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    pr_ci_path = os.path.join(save_dir, f"{model_name.replace(' ', '_')}_PR_with_CI.png")
    plt.savefig(pr_ci_path, dpi=300)
    plt.close()
    print(f"✅ Saved Precision-Recall curve with ribbon to: {pr_ci_path}")

# ================================
# 🚀 Run
# ================================
y_true_rf, y_prob_rf, auc_rf = evaluate_rf(df_rf, save_dir=save_dir)
plot_roc_with_ci(y_true_rf, y_prob_rf, save_dir=save_dir)
plot_pr_with_ci(y_true_rf, y_prob_rf, save_dir=save_dir)



📊 Random Forest Performance:
AUC: 0.997, Accuracy: 0.949, Precision: 0.930, Recall: 0.997, F1: 0.963
              precision    recall  f1-score   support

     No Fire       0.99      0.86      0.92       976
        Fire       0.93      1.00      0.96      1879

    accuracy                           0.95      2855
   macro avg       0.96      0.93      0.94      2855
weighted avg       0.95      0.95      0.95      2855

✅ Saved confusion matrix to: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Random_Forest_Confusion_Matrix.png
✅ Saved ROC curve with ribbon to: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Random_Forest_ROC_with_CI.png
✅ Saved Precision-Recall curve with ribbon to: /content/drive/MyDrive/California_Fire_MS/Fire_Risk_RF_Model_Performance/Random_Forest_PR_with_CI.png
